# OCI  RAG Agents

### What this file does:
Demonstrates using OCI Generative AI Agent Runtime service for out-of-the-box RAG with a shared knowledge base and citations.

**Documentation to reference:**
- OCI GenAI Agents: https://docs.oracle.com/en-us/iaas/Content/generative-ai-agents/home.htm
- Agent Runtime API: https://docs.oracle.com/en-us/iaas/api/#/en/generative-ai-agent-runtime/20240531/
- OCI Python SDK: https://github.com/oracle/oci-python-sdk/tree/master/src/oci/generative_ai_agent_runtime

**Relevant slack channels:**
- #generative-ai-agent-users or #igiu-innovation-lab: *For questions*
- #igiu-ai-learning: *For errors or help*

**Env setup:**
- sandbox.yaml: Needs "oci" (configFile, profile) and "agent" (endpoint) sections.
- .env: Load environment variables if needed.
- Knowledge base: Upload docs via OCI console (see rag_agents.md).
- configure cwd for jupyter match your workspace python code: 
    -  vscode menu -> Settings > Extensions > Jupyter > Notebook File Root
    -  change from `${fileDirname}` to `${workspaceFolder}`


**How to run in notebook:**
- Ensure dependencies installed (uv sync).
- Run cells in order.
- Update sandbox.yaml with your agent endpoint.

In [ ]:
# Import libraries
from oci.generative_ai_agent_runtime import GenerativeAiAgentRuntimeClient
import oci
import os
from dotenv import load_dotenv
from envyaml import EnvYAML

In [ ]:
# Constants
SANDBOX_CONFIG_FILE = "sandbox.yaml"
load_dotenv()
GENAI_URL = "https://agent-runtime.generativeai.us-chicago-1.oci.oraclecloud.com"

### Step 1: Load config and initialize the agent client

In [ ]:
# Load config
scfg = EnvYAML(SANDBOX_CONFIG_FILE)
if scfg is None or "oci" not in scfg or "bucket" not in scfg:
    raise RuntimeError("Invalid sandbox configuration.")

config = oci.config.from_file(os.path.expanduser(scfg["oci"]["configFile"]), scfg["oci"]["profile"])
endpoint = scfg["agent"]["endpoint"]

# Initialize agent client
llm_agent_client = GenerativeAiAgentRuntimeClient(
    config=config,
    service_endpoint=GENAI_URL,
    retry_strategy=oci.retry.NoneRetryStrategy(),
    timeout=(10, 240)
)

### Step 2: Create Session

In [ ]:
# Set up a session for conversational context.
# Create session details
create_session_details = oci.generative_ai_agent_runtime.models.CreateSessionDetails(
    display_name="Inno Lab C2M Agent",
    description="The agent has access to C2M documentation and can answer questions on it."
)

# Create session
create_session_response = llm_agent_client.create_session(create_session_details, endpoint)
session_id = create_session_response.data.id
print(f"Session created: {session_id}")

### Step 3: Ask Questions and Get Responses

In [ ]:
# Sample question 1
chat_detail = oci.generative_ai_agent_runtime.models.ChatDetails()
chat_detail.user_message = "what industries do we serve"
chat_detail.session_id = session_id

chat_response = llm_agent_client.chat(endpoint, chat_detail)
result = chat_response.data.message.content

print("****** Citation ******")
print(result.citations)
print("****** Answer ******")
print(result.text)

In [ ]:
# Sample question 2 (follow-up)
chat_detail = oci.generative_ai_agent_runtime.models.ChatDetails()
chat_detail.user_message = "tell me more about the second industry"
chat_detail.session_id = session_id

chat_response = llm_agent_client.chat(endpoint, chat_detail)
result = chat_response.data.message.content

print("****** Citation ******")
print(result.citations)
print("****** Answer ******")
print(result.text)

## Experimentation Section
Try these to learn more:
- Add documents to the knowledge base via OCI console.
- Ingest new documents and rerun queries.
- Switch to sessionless endpoint (update sandbox.yaml).
- Ask domain-specific questions (e.g., from uploaded docs).
- Try code-based queries (e.g., SQL or programming).

## Practice Exercises
1. **Add Documents**: Upload PDFs with images/charts to knowledge base and query.
2. **Ingest and Play**: Ingest docs, chat, and experiment with responses.
3. **Code Queries**: Try asking about code snippets or technical docs.
4. **Discussion**: How does agent RAG compare to home-grown RAG in ease and citations?

For help, reach out in #igiu-innovation-lab or #igiu-ai-learning.